<a href="https://colab.research.google.com/github/rajesh0305/Image-Processing-And-Computer-Vision-/blob/main/Transfer_Learning_With_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import wandb
from wandb.integration.keras import WandbMetricsLogger

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
# Sweep configuration
sweep_config = {
    'method': 'grid',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'batch_size': {'values': [8]},
        'learning_rate': {'values': [0.0001]},
        'img_size': {'values': [128]},  # Set a reasonably large size for MobileNetV2
        'epochs': {'values': [20]},
        'experiment': {'values': ['transfer_learning_with_fine_tune']}
    }
}

In [ ]:
# Initialize W&B sweep
sweep_id = wandb.sweep(sweep_config, project="5-flowers-transfer-learning-with-fine-tune")

Create sweep with ID: 7r0wbru8
Sweep URL: https://wandb.ai/rajesh8368568776-national-institute-of-technology-karnat/5-flowers-transfer-learning-with-fine-tune/sweeps/7r0wbru8


In [ ]:
# Train function
def train():
    with wandb.init() as run:
        config = wandb.config

        IMG_HEIGHT = config.img_size
        IMG_WIDTH = config.img_size
        IMG_CHANNELS = 3
        CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

        # Helper functions
        def read_and_decode(filename, resize_dims):
            img_bytes = tf.io.read_file(filename)
            img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
            img = tf.image.convert_image_dtype(img, tf.float32)
            img = tf.image.resize(img, resize_dims)
            return img

        def parse_csvline(csv_line):
            record_default = ["", ""]
            filename, label_string = tf.io.decode_csv(csv_line, record_default)
            img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
            label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]
            return img, label

        # Prepare datasets
        train_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        eval_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        # Build model
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
            include_top=False,    # Remove ImageNet head
            weights="imagenet"    # Use pretrained weights
        )
        base_model.trainable = True  # Freeze base

        model = keras.Sequential([
            base_model,
            keras.layers.GlobalAveragePooling2D(),
            keras.layers.BatchNormalization(),       # Add BatchNorm
            keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
        ])

        # Recompile with a smaller learning rate
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-5),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=["accuracy"]
        )

        # Visualize model
        model.summary()

        keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file="model_visualization.png")

        # Train
        callbacks = [WandbMetricsLogger(log_freq=5)]

        model.fit(
            train_dataset,
            validation_data=eval_dataset,
            epochs=config.epochs,
            callbacks=callbacks
        )

In [ ]:
# Launch W&B agent
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 6lmd6clf with config:
wandb: 	batch_size: 8
wandb: 	epochs: 20
wandb: 	experiment: transfer_learning_with_fine_tune
wandb: 	img_size: 128
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: rajesh8368568776 (rajesh8368568776-national-institute-of-technology-karnat) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         6,405 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,269,509 (8.66 MB)

 Trainable params: 2,232,837 (8.52 MB)

 Non-trainable params: 36,672 (143.25 KB)

Epoch 1/20
    413/Unknown 208s 449ms/step - accuracy: 0.4096 - loss: 1.5303

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 225s 491ms/step - accuracy: 0.4099 - loss: 1.5297 - val_accuracy: 0.5811 - val_loss: 1.1675
Epoch 2/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 200s 484ms/step - accuracy: 0.8333 - loss: 0.4902 - val_accuracy: 0.7270 - val_loss: 0.7818
Epoch 3/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 161s 388ms/step - accuracy: 0.9495 - loss: 0.2489 - val_accuracy: 0.7811 - val_loss: 0.6138
Epoch 4/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 202s 488ms/step - accuracy: 0.9905 - loss: 0.1388 - val_accuracy: 0.8162 - val_loss: 0.5286
Epoch 5/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 156s 376ms/step - accuracy: 0.9977 - loss: 0.0859 - val_accuracy: 0.8297 - val_loss: 0.4978
Epoch 6/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 172s 415ms/step - accuracy: 0.9988 - loss: 0.0579 - val_accuracy: 0.8378 - val_loss: 0.4815
Epoch 7/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 179s 432ms/step - accuracy: 1.0000 - loss: 0.0413 - val_accuracy: 0.8351 - val_loss: 0.4742
Epoch 8/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 201s 487ms/step - accuracy: 1.0000 - loss: 0.03

Click Here To See The Result Analysis:

https://wandb.ai/rajesh8368568776-national-institute-of-technology-karnat/5-flowers-transfer-learning-with-fine-tune/reports/Untitled-Report--VmlldzoxNTI4Nzc5OQ